<b><h1 style="text-align:center;">COMM493 - Coding AI for Business</h1></b>
<h5 style="text-align:center;">K-MEANS CLUSTERING FOR GYM MEMBERSHIP CUSTOMERS</h5>
<h5 style="text-align:center;">INSERT NAMES HERE</h5>
<h5 style="text-align:center;">March 24th, 2025</h5>

### Introduction
**Motivation:** TODO

**Data:** <a href="https://www.kaggle.com/datasets/ka66ledata/gym-membership-dataset/data">gym-membership-dataset</a> 

**Goal:** TODO

### 0: Set Up Environment
Load imports and define constants that will be used to prepare data and train model.

In [1]:
# Install the mxnet library required for certain functionalities in SageMaker
!pip install mxnet

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

# AWS SageMaker and Boto3
import sagemaker
from sagemaker import get_execution_role, KMeans
from sagemaker.image_uris import retrieve
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import boto3
from botocore.exceptions import ClientError

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Libraries
import tarfile
import mxnet as mx
import datetime
import json
import time
from tabulate import tabulate
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score

[03/24/25 20:39:22] INFO     Found credentials from IAM Role:                                   ]8;id=962086;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=839089;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24                                                                                             │
│   25 # Additional Libraries                                                                      │
│   26 import tarfile                                                                              │
│ ❱ 27 import mxnet as mx                                                                          │
│   28 import datetime                                                                             │
│   29 import json                                                                                 │
│   30 import time                                                                                 │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/mxnet/__init__.py:33 in       │
│ <module>                                                                                         │
│                                                                                                  │
│    30 # version info                                                                             │
│    31 __version__ = base.__version__                                                             │
│    32                                                                                            │
│ ❱  33 from . import contrib                                                                      │
│    34 from . import ndarray                                                                      │
│    35 from . import ndarray as nd                                                                │
│    36 from . import numpy                                                                        │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/mxnet/contrib/__init__.py:30  │
│ in <module>                                                                                      │
│                                                                                                  │
│   27 from . import autograd                                                                      │
│   28 from . import tensorboard                                                                   │
│   29                                                                                             │
│ ❱ 30 from . import text                                                                          │
│   31 from . import onnx                                                                          │
│   32 from . import io                                                                            │
│   33 from . import quantization                                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/mxnet/contrib/text/__init__.p │
│ y:23 in <module>                                                                                 │
│                                                                                                  │
│   20                                                                                             │
│   21 from . import utils                                                                         │
│   22 from . import vocab                                                                         │
│ ❱ 23 from . import embedding                               

### 1: SageMaker Setup and Configuration
ADD INFO HERE

In [ ]:
# Initialize a SageMaker Session
# A SageMaker Session helps in managing the interactions with the AWS SageMaker service, such as creating and training models, and deploying endpoints.
sagemaker_session = sagemaker.Session()

# Retrieve the IAM role
# The IAM role is required by SageMaker to access AWS resources (e.g., S3 buckets) on your behalf.
role = get_execution_role()

# Identify the AWS Region of the SageMaker Session
# This information is used to ensure that resources are created in the same region as the SageMaker session.
region = sagemaker_session.boto_region_name

# Create Boto3 Clients for S3 and SageMaker
# These clients enable programmatic access to S3 and SageMaker services for operations like uploading data and creating training jobs.
s3_client = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker')

# Retrieve the Docker image URI for the KMeans algorithm
# This URI is specific to the AWS region and is used to pull the appropriate image for training.
image_uri = retrieve('kmeans', region)

# Define the Default S3 Bucket for SageMaker
# SageMaker requires an S3 bucket to store training data, models, and outputs. Here, we use the session's default bucket.
bucket = sagemaker_session.default_bucket()

# Set the S3 Prefix for Storing Data
# The prefix is used to organize the data in the bucket, making it easier to manage and locate.
prefix = 'sagemaker/mall/kmeans-all'

# Output the IAM Role ARN and AWS Region for verification
print("Execution Role ARN:", role)
print("AWS Region:", region)

### 2: LOAD DATA SET
ADD INFO HERE

In [ ]:
df = pd.read_csv('gym_membership.csv')

print(df.head())